On import les bibliotheques :

In [ ]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import json

from pandas import json_normalize
import pandas as pd    
import json

import requests
import shutil
import os

import urllib.request

On crée la fonction de création du JSON

In [ ]:
####################################### CRÉATION JSON #################################################

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def crea_json():
    endpoint_url = "https://query.wikidata.org/sparql"

    query = """SELECT ?gateau ?gateauLabel ?gateauDescription ?image ?origineLabel
    {
    ?gateau wdt:P279/wdt:P31* wd:Q477248; #des pâtisseries
            wdt:P18 ?image; #l'image
            wdt:P495 ?origine. # l'origine du gateau
                
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en"}

    }
    LIMIT 1000"""

    results = get_results(endpoint_url, query)
    dico_json ={}

    for result in results["results"]["bindings"]:
        #print(result)
        # on récupère le nom du gateau
        gateau = result['gateauLabel']['value']
        # les valeurs qui nous interessent dans notre requête
        dico={}
        for key, val in result.items():
            dico[key]=val['value']
        dico_json[gateau]=dico
        #print("\n", gateau,'\n')
        #print(dico_json[gateau], '\n')

    #print(' dico json \n')
    #print(dico_json)
    with open('./fichier.json', 'w') as fichier :
        json.dump(dico_json, fichier)

crea_json()

On télécharge nos images

In [ ]:
def get_imgs():
    # the name of the json with all the data
    json_path="fichier.json"
    # we get the json
    json_data = json.load(open(json_path))


    for key, info in json_data.items():
        #print('\n', key,'\n', info, '\n')
        name = info['gateauLabel']# the name to save the image
        print(info['image'],"\n")
        urllib.request.urlretrieve(info['image'], "./script_python/images/"+name+".jpg")

get_imgs()

On cherche à récupérer les méta datas de chaque images

In [15]:
from PIL import Image
from PIL.ExifTags import TAGS 
#from exif import Image

def recup_matadata_img(path):
    dico_metadata ={}
    # the image's path
    #path =""

    # reading the image
    image = Image.open(path)
   
    #extracting the data
    exifdata = image.getexif()
    
    print(exifdata)
    # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    for tag_id in exifdata:
        # we get the tag name
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag

        # decoding the tags data
        if isinstance(data, bytes):
            data = data.decode()
        
        # we add that into the dictionnary
        dico_metadata[tag]= data
        print("tag : ", tag,"\n")
        print( data,"\n")
    return dico_metadata

recup_matadata_img("images/Alexandertorte.jpg")

AttributeError: has_exif

On cherche à sauvegarder ces datas

In [ ]:
import csv

def save_meta_data(path, indice):

    with open('meta_data.csv', 'w') as file:
        writer_obj=csv.writer(file)

        #for i in range(c.count_files('images')):# counting the number of images we have

        dico = recup.recup_matadata_img(path)#getting the meta_datas of the image
        header, values = [], []

        for key, value in dico.items():# the values to write into our csv file
                header.append(key)
                values.append(value)
            
        if indice ==0 :# we set the header of the file if it is the first call 
            writer_obj.writerows([ header, values ])# we save it in the csv file

        else :
            writer_obj.writerow(values)

save_meta_data("./script_python/images/Alexandertorte.jpg",0)